In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

# Function to compute annualized historical volatility (HV)
def compute_hv(ticker_obj, window=30):
    # Get recent historical data over a fixed period (60 days gives ample data)
    hist = ticker_obj.history(period='60d')
    # Compute the log returns: r_t = ln(Close_t / Close_{t-1})
    hist['LogReturn'] = np.log(hist['Close'] / hist['Close'].shift(1))
    # Drop missing values from first NaN after computing returns
    hist = hist.dropna()
    # The sample standard deviation of daily log returns multiplied by sqrt(252) to annualize
    hv = np.std(hist['LogReturn']) * np.sqrt(252)
    return hv, hist

# Function to benchmark option prices by comparing their IV to the computed HV
def benchmark_option_prices(symbol, expiration_date):
    # Load the ticker object using yfinance
    ticker_obj = yf.Ticker(symbol)
    
    # Compute the historical volatility (HV)
    hv, hist = compute_hv(ticker_obj)
    print("Historical Volatility (annualized): {:.2%}".format(hv))
    
    # Fetch the option chain data for the specified expiration date
    chain = ticker_obj.option_chain(expiration_date)
    # For demonstration we'll focus on calls. You can similarly process puts if desired.
    calls = chain.calls.copy()
    
    # Compute the mid price as the average of the bid and ask prices.
    # This is our benchmark price for the option.
    calls['mid'] = (calls['bid'] + calls['ask']) / 2
    
    # If the 'impliedVolatility' column is available, calculate the IV/HV ratio.
    if 'impliedVolatility' in calls.columns:
        # The IV/HV ratio helps assess overpricing:
        #   > 1 implies option IV is above historical levels (potentially overpriced),
        #   ≈ 1 indicates fair pricing,
        #   < 1 indicates IV is below historical levels.
        calls['IV/HV'] = calls['impliedVolatility'] / hv
    else:
        print("Implied Volatility data is not available in the chain.")
    
    # Sort by strike and display selected key columns.
    result = calls[['contractSymbol', 'strike', 'mid', 'impliedVolatility', 'IV/HV']].sort_values('strike')
    print(result)
    
if __name__ == "__main__":
    # Define your symbol. For example, use Apple Inc.
    symbol = "AAPL"
    
    # Instantiate the ticker object to fetch available option expiration dates.
    ticker_obj = yf.Ticker(symbol)
    expirations = ticker_obj.options
    print("Available Expirations: ", expirations)
    
    # Choose an expiration date for benchmarking (e.g., the first available)
    expiration_date = expirations[0]
    
    # Run the benchmark function
    benchmark_option_prices(symbol, expiration_date)


Available Expirations:  ('2025-04-11', '2025-04-17', '2025-04-25', '2025-05-02', '2025-05-09', '2025-05-16', '2025-05-23', '2025-06-20', '2025-07-18', '2025-08-15', '2025-09-19', '2025-10-17', '2025-12-19', '2026-01-16', '2026-03-20', '2026-06-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17')
Historical Volatility (annualized): 49.58%
         contractSymbol  strike     mid  impliedVolatility      IV/HV
0   AAPL250411C00100000   100.0  98.200           5.537112  11.168973
1   AAPL250411C00110000   110.0  88.875           2.875003   5.799201
2   AAPL250411C00120000   120.0  78.900           2.664066   5.373718
3   AAPL250411C00130000   130.0  69.400           3.201174   6.457125
4   AAPL250411C00140000   140.0  58.625           1.906250   3.845120
5   AAPL250411C00145000   145.0  53.575           3.225588   6.506371
6   AAPL250411C00150000   150.0  48.650           1.812501   3.656017
7   AAPL250411C00155000   155.0  43.500           0.000010   0.000020
8   AAPL250411C0016000